We will use those libraries in this tutorial:

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot

In [3]:
df_train=pd.read_csv('data/train.csv',header=0,index_col=0,parse_dates=['date'])
df_oil = pd.read_csv('data/oil.csv',header=0,parse_dates=['date'])
df_stores = pd.read_csv('data/stores.csv',header=0,index_col=0)
df_transactions = pd.read_csv('data/transactions.csv',header=0,parse_dates=['date'])
df_holidays = pd.read_csv('data/holidays_events.csv',header=0,parse_dates=['date'])

In [4]:
df_train.tail(10)

date  store_nbr                      family     sales  \
id                                                                    
3000878 2017-08-15          9                   MAGAZINES    11.000   
3000879 2017-08-15          9                       MEATS   449.228   
3000880 2017-08-15          9               PERSONAL CARE   522.000   
3000881 2017-08-15          9                PET SUPPLIES     6.000   
3000882 2017-08-15          9     PLAYERS AND ELECTRONICS     6.000   
3000883 2017-08-15          9                     POULTRY   438.133   
3000884 2017-08-15          9              PREPARED FOODS   154.553   
3000885 2017-08-15          9                     PRODUCE  2419.729   
3000886 2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3000887 2017-08-15          9                     SEAFOOD    16.000   

         onpromotion  
id                    
3000878            0  
3000879            0  
3000880           11  
3000881            0  
3000882            0  
3000883            0  
3000884            1  
3000885          148  
3000886            8  
3000887            0

In [5]:
df_oil.head(10)

date  dcoilwtico
0 2013-01-01         NaN
1 2013-01-02       93.14
2 2013-01-03       92.97
3 2013-01-04       93.12
4 2013-01-07       93.20
5 2013-01-08       93.21
6 2013-01-09       93.08
7 2013-01-10       93.81
8 2013-01-11       93.60
9 2013-01-14       94.27

As you can see, we have 43 missing values in oil dataset.

In [6]:
df_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [7]:
df_oil.iloc[0,1]=df_oil.iloc[1,1]
df_oil.fillna(method='ffill', inplace=True)
df_oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

Now, let's merge all the dataframes.

In [8]:
df_train = df_train.merge(df_oil, on='date', how='left')
df_train = df_train.merge(df_stores, on='store_nbr', how='left')
df_train = df_train.merge(df_transactions, on=['date','store_nbr'], how='left')
df_train = df_train.merge(df_holidays, on='date', how='left')

In [9]:
df_train

date  store_nbr                      family     sales  \
0       2013-01-01          1                  AUTOMOTIVE     0.000   
1       2013-01-01          1                   BABY CARE     0.000   
2       2013-01-01          1                      BEAUTY     0.000   
3       2013-01-01          1                   BEVERAGES     0.000   
4       2013-01-01          1                       BOOKS     0.000   
...            ...        ...                         ...       ...   
3054343 2017-08-15          9                     POULTRY   438.133   
3054344 2017-08-15          9              PREPARED FOODS   154.553   
3054345 2017-08-15          9                     PRODUCE  2419.729   
3054346 2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3054347 2017-08-15          9                     SEAFOOD    16.000   

         onpromotion  dcoilwtico   city      state type_x  cluster  \
0                  0       93.14  Quito  Pichincha      D       13   
1                  0       93.14  Quito  Pichincha      D       13   
2                  0       93.14  Quito  Pichincha      D       13   
3                  0       93.14  Quito  Pichincha      D       13   
4                  0       93.14  Quito  Pichincha      D       13   
...              ...         ...    ...        ...    ...      ...   
3054343            0       47.57  Quito  Pichincha      B        6   
3054344            1       47.57  Quito  Pichincha      B        6   
3054345          148       47.57  Quito  Pichincha      B        6   
3054346            8       47.57  Quito  Pichincha      B        6   
3054347            0       47.57  Quito  Pichincha      B        6   

         transactions   type_y    locale locale_name            description  \
0                 NaN  Holiday  National     Ecuador     Primer dia del ano   
1                 NaN  Holiday  National     Ecuador     Primer dia del ano   
2                 NaN  Holiday  National     Ecuador     Primer dia del ano   
3                 NaN  Holiday  National     Ecuador     Primer dia del ano   
4                 NaN  Holiday  National     Ecuador     Primer dia del ano   
...               ...      ...       ...         ...                    ...   
3054343        2155.0  Holiday     Local    Riobamba  Fundacion de Riobamba   
3054344        2155.0  Holiday     Local    Riobamba  Fundacion de Riobamba   
3054345        2155.0  Holiday     Local    Riobamba  Fundacion de Riobamba   
3054346        2155.0  Holiday     Local    Riobamba  Fundacion de Riobamba   
3054347        2155.0  Holiday     Local    Riobamba  Fundacion de Riobamba   

        transferred  
0             False  
1             False  
2             False  
3             False  
4             False  
...             ...  
3054343       False  
3054344       False  
3054345       False  
3054346       False  
3054347       False  

[3054348 rows x 16 columns]

In [10]:
df_plot=df_train[["date","sales"]].groupby(by='date',as_index=False).mean()
pyplot.plot(df_plot['date'],df_plot['sales'],label='Daily sales (2013-3017)')
pyplot.legend()
pyplot.show()

In [12]:
df_plot = df_train[["date","dcoilwtico","sales"]].groupby(by='date').mean()
df_plot.fillna(method='ffill',inplace=True)


# Initialize layout
fig, ax = pyplot.subplots(figsize = (9, 9))

# Add scatterplot
ax.scatter(df_plot['dcoilwtico'], df_plot['sales'], label='Daily sales vs oil price',
           s=60, alpha=0.7, edgecolors="k")

# Fit linear regression via least squares with numpy.polyfit
b3, b2, b1, a = np.polyfit(df_plot['dcoilwtico'], df_plot['sales'], deg=3)

# Create sequence of 100 numbers from 0 to 100 
xseq = np.linspace(20, 120, num=100)

# Plot regression line
ax.plot(xseq, a + b1 * xseq + b2 * xseq * xseq + b3 * xseq * xseq * xseq, color="k", lw=2.5)
ax.title.set_text('Daily sales vs oil price')
ax.set_xlabel('Oil price')
ax.set_ylabel('Daily sales')

Text(0, 0.5, 'Daily sales')

In [27]:
df_plot=df_train[["date","sales","onpromotion"]].groupby(by='date').mean()
df_plot['promotion']=0
#df_plot[df_plot.onpromotion>0]['promotion']=1
df_plot[df_plot.onpromotion>0]=df_plot[df_plot.onpromotion>0].assign(promotion=1)
#df_plot['promotion']=0 if df_plot['onpromotion']==0 else 1
df_plot.head(20)




sales  onpromotion  promotion
date                                          
2013-01-01    1.409438          0.0          0
2013-01-02  278.390807          0.0          0
2013-01-03  202.840197          0.0          0
2013-01-04  198.911154          0.0          0
2013-01-05  267.873244          0.0          0
2013-01-06  291.636028          0.0          0
2013-01-07  188.621100          0.0          0
2013-01-08  178.646340          0.0          0
2013-01-09  169.770375          0.0          0
2013-01-10  145.332213          0.0          0
2013-01-11  162.591294          0.0          0
2013-01-12  226.295293          0.0          0
2013-01-13  260.739926          0.0          0
2013-01-14  164.617487          0.0          0
2013-01-15  167.861700          0.0          0
2013-01-16  178.646416          0.0          0
2013-01-17  150.111401          0.0          0
2013-01-18  166.178928          0.0          0
2013-01-19  242.682296          0.0          0
2013-01-20  258.749936          0.0          0

In [29]:
df_plot=df_train[["date","sales","onpromotion"]].groupby(by='date').mean()
df_plot['promotion']=df_plot['onpromotion'].apply(lambda x: 1 if x>0 else 0)
df_plot.groupby(['promotion'])['promotion'].count()

# Initialize layout
fig, ax = plt.figure(figsize =(9, 9))
 
# Creating plot
plt.boxplot(df_plot['promotion'],df_plot['sales'])

promotion
0     454
1    1230
Name: promotion, dtype: int64